**✅ ETAPA 0 — Parâmetros iniciais**

In [1]:
# Definição dos principais parâmetros iniciais

# Tamanho do conjunto de treino, validação e teste (Etapa 2)
len_train = 60000
len_valid = 5000
len_test = 5000

# Tamanho máximo das sentenças (Etapa 4)
MAX_LEN = 35

# Tamanho máximo do vocabulário (Etapa 5)
max_vocab = 35000

# Tokens especiais (Etapa 5)
PAD_TOKEN = "<pad>"
SOS_TOKEN = "<sos>"
EOS_TOKEN = "<eos>"
UNK_TOKEN = "<unk>"

# Tamanho do batch (Etapa 7)
BATCH_SIZE = 64

# Dimensões das embeddings e do hidden state (Etapa 8)
EMB_DIM = 128
HIDDEN_DIM = 256

# Número de camadas e dropout para o Encoder e Decoder (Etapa 8)
var_num_layers = 1
var_dropout = 0.3

# Número de épocas e paciência para early stopping (Etapa 10)
N_EPOCHS = 15
PATIENCE = 3

# Taxa de aprendizado (Etapa 10)
var_lr = 0.001

# Parâmetros para a tradução com Beam Search (Etapa 12)
var_repetition_penalty = 0
var_beam_width = 5

**✅ ETAPA 1 — Download do Europarl EN–PT**

In [2]:
# Download do dataset de sentenças paralelas Europarl (Inglês-Português)
from datasets import load_dataset

dataset = load_dataset(
    "sentence-transformers/parallel-sentences-europarl",
    "en-pt"
)

d:\Natural_Language_Processing\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Imports necessários
import torch
import re
import random
from collections import Counter
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch.nn as nn
import time
import nltk
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import corpus_bleu
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to C:\Users\Carlos
[nltk_data]     Soares\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Carlos
[nltk_data]     Soares\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
# Configurando o dispositivo para treinamento (GPU se disponível)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


**✅ ETAPA 2 — Inspeção + Amostragem do Europarl EN–PT**

In [5]:
# Conferindo o conteúdo do dataset
dataset.keys()

dict_keys(['train'])

In [6]:
# Conferindo a primeira amostra do conjunto de treino
print(dataset["train"][0])

{'english': 'Resumption of the session', 'non_english': 'Reinício da sessão'}


In [7]:
# Embaralhando e criando subconjuntos menores para treino, validação e teste
dataset_shuffled = dataset["train"].shuffle(seed=42)

dataset_small = {
    "train": dataset_shuffled.select(range(0, len_train)),
    "validation": dataset_shuffled.select(range(len_train, len_train + len_valid)),
    "test": dataset_shuffled.select(range(len_train + len_valid, len_train + len_valid + len_test)),
}

In [8]:
# Conferindo o tamanho dos novos subconjuntos
for split in dataset_small:
    print(split, len(dataset_small[split]))

train 60000
validation 5000
test 5000


In [9]:
# Conferindo uma amostra do conjunto de treino
sample = dataset_small["train"][0]
print("EN:", sample["english"])
print("PT:", sample["non_english"])

EN: It is not enough to enable them to get the hang of the programmes and the management structures as well, and then require only 3500 posts up to 2008, all of which is described as 'enlargement costs'.
PT: O facto de se autorizar um alargamento dos programas e também das estruturas administrativas, e apenas se exigir a criação de 3 500 novos lugares até 2008, que será designado como o custo do alargamento, parece naturalmente muito pouco.


**✅ ETAPA 3 — Limpeza mínima de texto (controlada)**

In [10]:
# Função de pré-processamento de texto (convertendo para minúsculas e removendo espaços extras)
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\s+", " ", text)
    text = text.strip()
    return text

In [11]:
# Aplicando o pré-processamento ao dataset
def preprocess(example):
    return {
        "en": clean_text(example["english"]),
        "pt": clean_text(example["non_english"])
    }

dataset_clean = {}

for split in dataset_small:
    dataset_clean[split] = dataset_small[split].map(
        preprocess,
        remove_columns=["english", "non_english"]
    )

In [12]:
# Conferindo uma amostra do conjunto de treino antes e depois do pré-processamento
print("ANTES:")
print(dataset_small["train"][0])

print("\nDEPOIS:")
print(dataset_clean["train"][0])

ANTES:
{'english': "It is not enough to enable them to get the hang of the programmes and the management structures as well, and then require only 3500 posts up to 2008, all of which is described as 'enlargement costs'.", 'non_english': 'O facto de se autorizar um alargamento dos programas e também das estruturas administrativas, e apenas se exigir a criação de 3 500 novos lugares até 2008, que será designado como o custo do alargamento, parece naturalmente muito pouco.'}

DEPOIS:
{'en': "it is not enough to enable them to get the hang of the programmes and the management structures as well, and then require only 3500 posts up to 2008, all of which is described as 'enlargement costs'.", 'pt': 'o facto de se autorizar um alargamento dos programas e também das estruturas administrativas, e apenas se exigir a criação de 3 500 novos lugares até 2008, que será designado como o custo do alargamento, parece naturalmente muito pouco.'}


**✅ ETAPA 4 — Filtrar frases vazias e muito longas**

In [13]:
# Criando função para filtrar sentenças muito longas ou vazias
def length_filter(example):
    return (
        len(example["en"].split()) > 0 and
        len(example["pt"].split()) > 0 and
        len(example["en"].split()) <= MAX_LEN and
        len(example["pt"].split()) <= MAX_LEN
    )

In [14]:
# Filtrando o dataset
dataset_filtered = {}

for split in dataset_clean:
    dataset_filtered[split] = dataset_clean[split].filter(length_filter)

In [15]:
# Conferindo o tamanho dos datasets antes e depois da filtragem
for split in dataset_clean:
    print(
        split,
        "antes:", len(dataset_clean[split]),
        "depois:", len(dataset_filtered[split])
    )

train antes: 60000 depois: 45307
validation antes: 5000 depois: 3790
test antes: 5000 depois: 3733


**✅ ETAPA 5 — Construção do vocabulário (word-level)**

In [16]:
# Construindo vocabulários para inglês e português
def build_vocab(sentences, max_vocab_size=max_vocab):
    counter = Counter()

    for sent in sentences:
        counter.update(sent.split())

    vocab = {
        PAD_TOKEN: 0,
        SOS_TOKEN: 1,
        EOS_TOKEN: 2,
        UNK_TOKEN: 3,
    }

    for word, _ in counter.most_common(max_vocab_size - len(vocab)):
        vocab[word] = len(vocab)

    return vocab

In [17]:
# Construindo os vocabulários
en_vocab = build_vocab(dataset_filtered["train"]["en"])
pt_vocab = build_vocab(dataset_filtered["train"]["pt"])

In [18]:
# Conferindo o tamanho dos vocabulários
print("EN vocab size:", len(en_vocab))
print("PT vocab size:", len(pt_vocab))

EN vocab size: 35000
PT vocab size: 35000


In [19]:
# Conferindo uma amostra do conjunto de treino e sua representação numérica
sample = dataset_filtered["train"][0]["en"]
print(sample)
print([en_vocab.get(tok, en_vocab[UNK_TOKEN]) for tok in sample.split()])

i wish to make four points.
[13, 240, 6, 89, 602, 1281]


**✅ ETAPA 6 — Encoding das frases + Dataset PyTorch**

In [20]:
# Função para converter sentenças em sequências de índices, incluindo tokens especiais
def encode_sentence(sentence, vocab):
    tokens = sentence.split()
    return (
        [vocab[SOS_TOKEN]] +
        [vocab.get(tok, vocab[UNK_TOKEN]) for tok in tokens] +
        [vocab[EOS_TOKEN]]
    )

In [21]:
# Aplicando a codificação ao dataset
def encode_example(example):
    return {
        "en_ids": encode_sentence(example["en"], en_vocab),
        "pt_ids": encode_sentence(example["pt"], pt_vocab),
    }

dataset_encoded = {}

for split in dataset_filtered:
    dataset_encoded[split] = dataset_filtered[split].map(encode_example)


In [22]:
# Conferindo uma amostra do conjunto de treino codificado
sample = dataset_encoded["train"][0]

print("EN texto:", dataset_filtered["train"][0]["en"])
print("EN ids:", sample["en_ids"])

print("PT texto:", dataset_filtered["train"][0]["pt"])
print("PT ids:", sample["pt_ids"])

EN texto: i wish to make four points.
EN ids: [1, 13, 240, 6, 89, 602, 1281, 2]
PT texto: gostaria de focar quatro aspectos.
PT ids: [1, 55, 4, 4652, 659, 2890, 2]


In [23]:
# Criando uma classe Dataset personalizada para PyTorch
class TranslationDataset(Dataset):
    def __init__(self, hf_dataset):
        self.data = hf_dataset

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "src": torch.tensor(self.data[idx]["en_ids"], dtype=torch.long),
            "tgt": torch.tensor(self.data[idx]["pt_ids"], dtype=torch.long),
        }

In [24]:
# Criando instâncias dos datasets para PyTorch
train_dataset = TranslationDataset(dataset_encoded["train"])
val_dataset   = TranslationDataset(dataset_encoded["validation"])
test_dataset  = TranslationDataset(dataset_encoded["test"])

In [25]:
# Conferindo uma amostra do conjunto de treino PyTorch
sample = train_dataset[0]
print(sample["src"])
print(sample["tgt"])

tensor([   1,   13,  240,    6,   89,  602, 1281,    2])
tensor([   1,   55,    4, 4652,  659, 2890,    2])


**✅ ETAPA 7 — collate_fn + DataLoader (padding correto)**

In [26]:
# Função de colagem para DataLoader do PyTorch
def collate_fn(batch):
    src_batch = [item["src"] for item in batch]
    tgt_batch = [item["tgt"] for item in batch]

    src_padded = pad_sequence(
        src_batch,
        batch_first=True,
        padding_value=en_vocab[PAD_TOKEN]
    )

    tgt_padded = pad_sequence(
        tgt_batch,
        batch_first=True,
        padding_value=pt_vocab[PAD_TOKEN]
    )

    return src_padded, tgt_padded

In [27]:
# Criando DataLoaders para treino, validação e teste
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn
)

**✅ ETAPA 8 — Implementação do Encoder (LSTM)**

In [28]:
# Criando a classe Encoder usando LSTM em PyTorch
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, num_layers=var_num_layers, dropout=var_dropout):
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=input_dim,
            embedding_dim=emb_dim,
            padding_idx=en_vocab[PAD_TOKEN]
        )
        
        self.dropout = nn.Dropout(dropout)

        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        encoder_outputs, (hidden, cell) = self.lstm(embedded)
        
        return encoder_outputs, hidden, cell

In [29]:
# Criando uma instância do Encoder
INPUT_DIM = len(en_vocab)
# EMB_DIM definidio anteriormente
# HIDDEN_DIM definidio anteriormente

encoder = Encoder(
    input_dim=INPUT_DIM,
    emb_dim=EMB_DIM,
    hidden_dim=HIDDEN_DIM
)

In [30]:
# Testando o Encoder com um batch de dados
src, tgt = next(iter(train_loader))
src = src.to(device)
tgt = tgt.to(device)

encoder = encoder.to(device)

encoder_outputs, hidden, cell = encoder(src)

print("encoder_outputs:", encoder_outputs.shape, encoder_outputs.device)
print("hidden:", hidden.shape, hidden.device)
print("cell:", cell.shape, cell.device)

encoder_outputs: torch.Size([64, 37, 256]) cuda:0
hidden: torch.Size([1, 64, 256]) cuda:0
cell: torch.Size([1, 64, 256]) cuda:0


**✅ ETAPA 8.1 — Implementação do Attention**

In [31]:
# Criando a classe de Atenção Bahdanau em PyTorch
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        batch_size = encoder_outputs.size(0)
        src_len = encoder_outputs.size(1)

        # Repetir hidden para cada timestep do encoder
        hidden = hidden.permute(1, 0, 2)              # [batch, 1, hidden]
        hidden = hidden.repeat(1, src_len, 1)         # [batch, src_len, hidden]

        # Concatenar hidden do decoder com outputs do encoder
        energy = torch.tanh(
            self.attn(torch.cat((hidden, encoder_outputs), dim=2))
        )                                              # [batch, src_len, hidden]

        # Calcular scores
        attention = self.v(energy).squeeze(2)         # [batch, src_len]

        # Softmax para obter pesos
        attention_weights = torch.softmax(attention, dim=1)

        # Context vector = soma ponderada
        context = torch.bmm(
            attention_weights.unsqueeze(1),
            encoder_outputs
        ).squeeze(1)                                   # [batch, hidden]

        return context, attention_weights

In [32]:
# Testando a Atenção Bahdanau com saídas do Encoder
attention = BahdanauAttention(HIDDEN_DIM).to(device)

src, tgt = next(iter(train_loader))
src = src.to(device)

encoder_outputs, hidden, cell = encoder(src)

context, attn_weights = attention(hidden, encoder_outputs)

print("context:", context.shape)
print("attn_weights:", attn_weights.shape)

context: torch.Size([64, 256])
attn_weights: torch.Size([64, 37])


**✅ ETAPA 9 — Implementação do Decoder (LSTM + Teacher Forcing)**

In [33]:
# Criando a classe Decoder usando LSTM em PyTorch
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, attention, dropout=0.3):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.dropout = nn.Dropout(dropout)

        self.lstm = nn.LSTM(
            emb_dim + hidden_dim,
            hidden_dim,
            batch_first=True
        )

        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, input, hidden, cell, encoder_outputs):
        input = input.unsqueeze(1)
        embedded = self.dropout(self.embedding(input))

        context, attention_weights = self.attention(hidden, encoder_outputs)
        context = context.unsqueeze(1)

        lstm_input = torch.cat((embedded, context), dim=2)
        output, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))

        prediction = self.fc_out(output.squeeze(1))

        return prediction, hidden, cell, attention_weights

In [34]:
# Criando uma instância do Decoder
attention = BahdanauAttention(HIDDEN_DIM).to(device)
decoder = Decoder(len(pt_vocab), EMB_DIM, HIDDEN_DIM, attention).to(device)

src, tgt = next(iter(train_loader))
src = src.to(device)
tgt = tgt.to(device)

encoder_outputs, hidden, cell = encoder(src)

input_token = tgt[:, 0]  # <sos>

output, hidden, cell, attn_weights = decoder(
    input_token,
    hidden,
    cell,
    encoder_outputs
)

print("output:", output.shape)
print("attn_weights:", attn_weights.shape)

output: torch.Size([64, 35000])
attn_weights: torch.Size([64, 35])


**✅ ETAPA 10 — Classe Seq2Seq + Loop de Treino (Teacher Forcing)**

In [35]:
# Criando a classe Seq2Seq combinando Encoder e Decoder
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        tgt_len = tgt.size(1)
        tgt_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(
            batch_size,
            tgt_len,
            tgt_vocab_size
        ).to(self.device)

        encoder_outputs, hidden, cell = self.encoder(src)

        input = tgt[:, 0]  # <sos>

        for t in range(1, tgt_len):
            output, hidden, cell, _ = self.decoder(
                input,
                hidden,
                cell,
                encoder_outputs
            )

            outputs[:, t] = output

            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)

            input = tgt[:, t] if teacher_force else top1

        return outputs

In [36]:
# Criando uma instância do modelo Seq2Seq
model = Seq2Seq(encoder, decoder, device).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=pt_vocab[PAD_TOKEN])
optimizer = torch.optim.Adam(model.parameters(), lr=var_lr)

**✅ ETAPA 11 — Treinamento do modelo**

In [37]:
# Função de treinamento para uma época
def train_epoch(model, dataloader, optimizer, criterion, clip=1.0):
    model.train()
    epoch_loss = 0

    for src, tgt in dataloader:
        src = src.to(device)
        tgt = tgt.to(device)

        optimizer.zero_grad()

        output = model(src, tgt)
        output_dim = output.shape[-1]

        output = output[:, 1:].reshape(-1, output_dim)
        tgt = tgt[:, 1:].reshape(-1)

        loss = criterion(output, tgt)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

In [38]:
# Função para traduzir uma sentença usando beam search
def translate_sentence_beam(
    sentence,
    src_vocab,
    tgt_vocab,
    model,
    device,
    max_len=40,
    beam_width=5,
    repetition_penalty=1.2
):
    model.eval()
    
    # Codificar sentença source
    tokens = sentence.lower().split()
    src_indexes = (
        [src_vocab["<sos>"]] +
        [src_vocab.get(tok, src_vocab["<unk>"]) for tok in tokens] +
        [src_vocab["<eos>"]]
    )
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    with torch.no_grad():
        encoder_outputs, hidden, cell = model.encoder(src_tensor)
    
    # Inicializar beam com <sos>
    beams = [(
        [tgt_vocab["<sos>"]],  # sequência
        0.0,                    # score acumulado
        hidden,                 # hidden state
        cell                    # cell state
    )]
    
    completed = []
    
    for _ in range(max_len):
        all_candidates = []
        
        for seq, score, h, c in beams:
            # Se já terminou, adiciona aos completos
            if seq[-1] == tgt_vocab["<eos>"]:
                completed.append((seq, score))
                continue
            
            # Próximo token
            tgt_tensor = torch.LongTensor([seq[-1]]).to(device)
            
            with torch.no_grad():
                output, new_h, new_c, _ = model.decoder(
                    tgt_tensor,
                    h,
                    c,
                    encoder_outputs
                )
            
            # Top-k probabilidades
            log_probs = torch.log_softmax(output, dim=1)

            for token_id in set(seq):  # Tokens já usados
                if token_id < log_probs.size(1):
                    log_probs[0, token_id] /= repetition_penalty

            topk_probs, topk_indices = torch.topk(log_probs, beam_width)
            
            # Criar candidatos
            for i in range(beam_width):
                token = topk_indices[0, i].item()
                token_score = topk_probs[0, i].item()
                
                new_seq = seq + [token]
                new_score = score + token_score
                
                all_candidates.append((new_seq, new_score, new_h, new_c))
        
        # Selecionar top-k beams
        beams = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_width]
        
        # Se todos completaram, para
        if len(completed) >= beam_width:
            break
    
    # Adicionar beams não terminados aos completos
    for seq, score, _, _ in beams:
        if seq[-1] != tgt_vocab["<eos>"]:
            completed.append((seq, score))
    
    # Escolher melhor sequência (normalizado pelo tamanho)
    if not completed:
        best_seq = beams[0][0]
    else:
        best_seq = max(completed, key=lambda x: x[1] / len(x[0]))[0]
    
    # Converter para palavras
    tgt_vocab_inv = {v: k for k, v in tgt_vocab.items()}
    tgt_tokens = [
        tgt_vocab_inv[idx]
        for idx in best_seq
        if idx not in (
            tgt_vocab["<sos>"],
            tgt_vocab["<eos>"],
            tgt_vocab["<pad>"]
        )
    ]
    
    return " ".join(tgt_tokens)

In [39]:
# Função de avaliação usando BLEUScore
def calculate_bleu(model, dataset, src_vocab, tgt_vocab, device, num_samples=100):
        
    model.eval()
    references = []
    hypotheses = []
    
    # Vocabulário invertido
    src_vocab_inv = {v: k for k, v in src_vocab.items()}
    tgt_vocab_inv = {v: k for k, v in tgt_vocab.items()}
    
    for i in range(min(num_samples, len(dataset))):
        sample = dataset[i]
        
        # Reconstruir sentença source
        src_tokens = [src_vocab_inv[idx] for idx in sample['en_ids'] 
                      if idx not in [src_vocab['<sos>'], src_vocab['<eos>'], src_vocab['<pad>']]]
        src_sentence = " ".join(src_tokens)
        
        # Referência (tradução correta)
        ref_tokens = [tgt_vocab_inv[idx] for idx in sample['pt_ids']
                      if idx not in [tgt_vocab['<sos>'], tgt_vocab['<eos>'], tgt_vocab['<pad>']]]
        
        # Tradução do modelo
        translation = translate_sentence_beam(src_sentence, src_vocab, tgt_vocab, model, device)
        
        references.append([ref_tokens])
        hypotheses.append(translation.split())
    
    return corpus_bleu(references, hypotheses) * 100

In [40]:
# Função de avaliação usando METEOR
def calculate_meteor(model, dataset, src_vocab, tgt_vocab, device, num_samples=100):
    model.eval()
    scores = []
    
    # Vocabulário invertido
    src_vocab_inv = {v: k for k, v in src_vocab.items()}
    tgt_vocab_inv = {v: k for k, v in tgt_vocab.items()}
    
    for i in range(min(num_samples, len(dataset))):
        sample = dataset[i]
        
        # Reconstruir sentença source
        src_tokens = [src_vocab_inv[idx] for idx in sample['en_ids'] 
                      if idx not in [src_vocab['<sos>'], src_vocab['<eos>'], src_vocab['<pad>']]]
        src_sentence = " ".join(src_tokens)
        
        # Referência
        ref_tokens = [tgt_vocab_inv[idx] for idx in sample['pt_ids']
                      if idx not in [tgt_vocab['<sos>'], tgt_vocab['<eos>'], tgt_vocab['<pad>']]]
        
        # Tradução
        translation = translate_sentence_beam(src_sentence, src_vocab, tgt_vocab, model, device)
        
        # METEOR espera lista de tokens
        score = meteor_score([ref_tokens], translation.split())
        scores.append(score)
    
    return sum(scores) / len(scores) * 100

In [41]:
# Função de validação
best_val_loss = float('inf')
patience_counter = 0

def eval_epoch(model, dataloader, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for src, tgt in dataloader:
            src = src.to(device)
            tgt = tgt.to(device)
            
            output = model(src, tgt, teacher_forcing_ratio=0)
            
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            tgt = tgt[:, 1:].reshape(-1)
            
            loss = criterion(output, tgt)
            epoch_loss += loss.item()
    
    return epoch_loss / len(dataloader)

for epoch in range(N_EPOCHS):
    start = time.time()
    
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    val_loss = eval_epoch(model, val_loader, criterion)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pt')
        print("✓ Modelo salvo")
    else:
        patience_counter += 1
    
    print(
        f"Epoch {epoch+1}/{N_EPOCHS} | "
        f"Train: {train_loss:.3f} | Val: {val_loss:.3f} | "
        f"Time: {time.time()-start:.1f}s"
    )
    
    if patience_counter >= PATIENCE:
        print(f"\nEarly stopping após {epoch+1} épocas")
        break

✓ Modelo salvo
Epoch 1/15 | Train: 6.902 | Val: 6.521 | Time: 365.1s
✓ Modelo salvo
Epoch 2/15 | Train: 6.211 | Val: 6.265 | Time: 365.2s
✓ Modelo salvo
Epoch 3/15 | Train: 5.831 | Val: 6.048 | Time: 362.3s
✓ Modelo salvo
Epoch 4/15 | Train: 5.547 | Val: 5.943 | Time: 362.0s
✓ Modelo salvo
Epoch 5/15 | Train: 5.324 | Val: 5.863 | Time: 362.1s
✓ Modelo salvo
Epoch 6/15 | Train: 5.130 | Val: 5.806 | Time: 368.6s
✓ Modelo salvo
Epoch 7/15 | Train: 4.955 | Val: 5.741 | Time: 367.3s
✓ Modelo salvo
Epoch 8/15 | Train: 4.793 | Val: 5.723 | Time: 364.4s
✓ Modelo salvo
Epoch 9/15 | Train: 4.651 | Val: 5.717 | Time: 365.0s
✓ Modelo salvo
Epoch 10/15 | Train: 4.527 | Val: 5.701 | Time: 364.6s
✓ Modelo salvo
Epoch 11/15 | Train: 4.408 | Val: 5.681 | Time: 385.9s
✓ Modelo salvo
Epoch 12/15 | Train: 4.297 | Val: 5.636 | Time: 385.3s
Epoch 13/15 | Train: 4.185 | Val: 5.693 | Time: 388.9s
Epoch 14/15 | Train: 4.109 | Val: 5.650 | Time: 378.1s
Epoch 15/15 | Train: 3.996 | Val: 5.687 | Time: 371.0s

Ear

In [42]:
# Carregar melhor modelo e calcular métricas
model.load_state_dict(torch.load('best_model.pt'))
bleu = calculate_bleu(model, dataset_encoded['test'], en_vocab, pt_vocab, device)
meteor = calculate_meteor(model, dataset_encoded['test'], en_vocab, pt_vocab, device)

print(f"BLEU Score: {bleu:.2f}")
print(f"METEOR Score: {meteor:.2f}")

BLEU Score: 10.36
METEOR Score: 27.01


**✅ ETAPA 12 — Função de Inferência (Greedy Decoding)**

In [46]:
# Invertendo o vocabulário de português para mapeamento índice -> token
src_vocab = en_vocab
tgt_vocab = pt_vocab
tgt_vocab_inv = {v: k for k, v in tgt_vocab.items()}

test_sentences_europarl = [
    # Frases curtas e comuns
    "i support this proposal",
    "the report is very important",
    "we must take action now",
    "i voted for this report",
    "the commission should act",
    
    # Estruturas típicas do Europarl
    "this is a good report",
    "we need more time",
    "the situation is serious",
    "i thank the rapporteur",
    "we have to work together",
    
    # Frases sobre política/economia (domínio do dataset)
    "we need economic growth",
    "human rights are important",
    "climate change is a problem",
    "the budget must be approved",
    "member states should cooperate",

    # Frases fora do domínio (desafio extra)
    "good morning",
    "hello world"
]

print("=== Traduções com Beam Search ===\n")
for sentence in test_sentences_europarl:
    translation = translate_sentence_beam(
        sentence,
        src_vocab,
        tgt_vocab,
        model,
        device,
        beam_width=var_beam_width,
        repetition_penalty=var_repetition_penalty
    )
    print(f"EN: {sentence}")
    print(f"PT: {translation}")
    print()

=== Traduções com Beam Search ===

EN: i support this proposal
PT: por esta proposta de resolução.

EN: the report is very important
PT: o relatório é muito importante.

EN: we must take action now
PT: temos de tomar agora a avançar.

EN: i voted for this report
PT: votei a favor deste relatório.

EN: the commission should act
PT: a comissão deve <unk>

EN: this is a good report
PT: trata-se de um relatório <unk>

EN: we need more time
PT: precisamos de mais

EN: the situation is serious
PT: a situação <unk>

EN: i thank the rapporteur
PT: agradeço ao relator

EN: we have to work together
PT: temos de trabalhar em conjunto.

EN: we need economic growth
PT: precisamos de crescimento económico.

EN: human rights are important
PT: os direitos humanos são importantes.

EN: climate change is a problem
PT: as alterações climáticas é um problema problema.

EN: the budget must be approved
PT: o orçamento deve ser

EN: member states should cooperate
PT: os estados-membros devem ser

EN: good mo